This notebook demonstrates some basic techniques for working with LIDAR data in Jupyter.

Resources:
- Breddels, Maarten. “Interactive 3D Visualization in Jupyter.” Presented at the SciPy 2018, February 4, 2018. [https://www.youtube.com/watch?v=hOKa8klJPyo](https://www.youtube.com/watch?v=hOKa8klJPyo).
- Carette, Mathieu. “Efficient and Interactive 3D Point Cloud Processing.” Presented at the FOSDEM 2018, February 4, 2018. [https://www.youtube.com/watch?v=1oXVDG6Iop0](https://www.youtube.com/watch?v=1oXVDG6Iop0). [notebook](https://github.com/rockestate/point-cloud-processing/blob/master/notebooks/point-cloud-processing-static.ipynb)




## Pointcloud Classes

from [https://github.com/CityOfNewYork/nyc-geo-metadata/blob/master/Metadata/Metadata_TopobathymetricClassifiedPointCloud.md]

- 1 - Default/Unclassified
- 1 WO - Default/Unclassified Withheld Overlap
- 2 - Ground
- 7 W - Noise Withheld
- 9 - Water
- 10 - Ignored Ground (Water's Edge)
- 17 - Bridge
- 25 - Subway Stairs
- 40 - Bathymetric Bottom
- 41 - Water Surface
- 45 - Water Column

In [19]:
from dotenv import load_dotenv, find_dotenv
from geoalchemy2 import Geometry, WKTElement
import geopandas as gpd
import ipyleaflet
import ipyvolume as ipv
import json
import numpy as np
import pandas as pd
import pdal
import pyproj
import pythreejs
import scipy
from shapely.geometry import shape
from shapely.ops import transform
from sqlalchemy import *


In [2]:
# query postgis to retrieve points
load_dotenv(find_dotenv())
conn_vars = ['DEV_USER', 'DEV_PASSWORD', 'DEV_HOST', 'DEV_PORT', 'DEV_DB']
user, password, host, port, dbname = [os.getenv(var) for var in conn_vars]
conn_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'

In [3]:
engine = create_engine(conn_string)

In [4]:
m = ipyleaflet.Map(center=(40.712160, -74.007153), zoom=16)
dc = ipyleaflet.DrawControl()
m.add_control(dc)
m

Map(center=[40.71216, -74.007153], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [82]:
# get site bounds reprojected as epsg 2263
project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:2263')
)

map_bounds = dc.last_draw['geometry']
site_bounds = transform(project.transform, shape(map_bounds)).wkt

/Users/carstenrodin/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/carstenrodin/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/carstenrodin/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<c

In [91]:
mksize 

NameError: name 'mksize' is not defined

In order to get eigenvalues and normals needed for classification and mesh separation, the points need to be imported through PDAL rather than straight from PostGIS to a dataframe:

In [83]:
# get lidar points using PDAL
pipeline_def = {
    "pipeline": [
        {
            "type":"readers.pgpointcloud",
            "connection": f'host={host} dbname={dbname} user={user} password={password} port={port}',
            "table": "doitt_pointcloud_v2017",
            "column": "pa",
            "spatialreference": "EPSG:2263",
            "where": f"PC_Intersects(pa, ST_GeomFromText('{site_bounds}',2263))"
        },
        {
            "type":"filters.crop",
            "polygon": f"{site_bounds}",
            "distance": 500
        },
        {   "type":"filters.hag"},
        {   "type":"filters.eigenvalues",
            "knn":16},
        {   "type":"filters.normal",
            "knn":16}
    ]}
pipeline = pdal.Pipeline(json.dumps(pipeline_def))
pipeline.validate()
%time n_points = pipeline.execute()


CPU times: user 76.4 ms, sys: 15.4 ms, total: 91.8 ms
Wall time: 361 ms


In [84]:
arr = pipeline.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})

In [85]:
# "zero" all coordinates for visualization
df['X_0'] = df['X'] - df['X'].min()
df['Y_0'] = df['Y'] - df['Y'].min()
df['Z_0'] = df['Z'] - df['Z'].min()


In [87]:
# z and y coordinates are swapped here to use the orientation convention of ipyvolume
fig = ipv.figure()
fig.camera_control = 'pan'

control = pythreejs.OrbitControls(controlling=fig.camera)
fig.controls = control
# control.autoRotate = True

scatter = ipv.scatter(
    df['X_0'].to_numpy(),
    df['Z_0'].to_numpy(), 
    df['Y_0'].to_numpy(), 
    marker='box', 
    size=mksize)

ipv.squarelim()
ipv.style.box_off()
ipv.style.axes_off()
ipv.show()

In [88]:
# filter out trees & vegetation
df['tree'] = (df['Classification']==1) & (df['HeightAboveGround'] >= 2) & (df['Eigenvalue0'] > .05) &  (df['NumberOfReturns'] - df['ReturnNumber'] >= 1)

tree = ipv.scatter(
    df.loc[df['tree'], 'X_0'].to_numpy(),
    df.loc[df['tree'], 'Z_0'].to_numpy(),
    df.loc[df['tree'], 'Y_0'].to_numpy(),
    marker='box', 
    size=mksize)

nontree = ipv.scatter(
    df.loc[-df['tree'], 'X_0'].to_numpy(),
    df.loc[-df['tree'], 'Z_0'].to_numpy(),
    df.loc[-df['tree'], 'Y_0'].to_numpy(),
    marker='box', 
    size=mksize)

tree.color='darkgreen'
nontree.color='lightgrey'

# turn off the original one
scatter.visible=False


In [89]:
# get roof points
roof_mask = (df['Classification'] == 1) & (df['HeightAboveGround'] > 7) & (df['Eigenvalue0'] <= .02) & (df['NumberOfReturns'] == df['ReturnNumber'])

In [90]:
roof_normals = ipv.quiver(
    df.loc[roof_mask, 'X_0'].to_numpy(),
    df.loc[roof_mask, 'Z_0'].to_numpy(),
    df.loc[roof_mask, 'Y_0'].to_numpy(),
    df.loc[roof_mask, 'NormalX'].to_numpy(),
    df.loc[roof_mask, 'NormalZ'].to_numpy(),
    df.loc[roof_mask, 'NormalY'].to_numpy(),
    size=3)
fig.scatters.append(roof_normals)

In [41]:
# mark points as ground/nonground, construct ground surface
df['ground'] = df['Classification'] == 2
ground_mesh = scipy.spatial.Delaunay(df.loc[df['ground'], ['X', 'Y']])

In [ ]:
# process the results to get a "section"

## More Resources

- [Point Data Abstraction Library (PDAL)](https://pdal.io/index.html)
- [PostgreSQL Pointcloud Extension](https://github.com/pgpointcloud/pointcloud)